# RAPIDS cuDF

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ritchieng/deep-learning-wizard/blob/master/docs/machine_learning/gpu/rapids_cudf.ipynb)

## Environment Setup

### Check Version

#### Python Version

In [1]:
# Check Python Version
!python --version

Python 3.6.7


#### Ubuntu Version

In [2]:
# Check Ubuntu Version
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.2 LTS
Release:	18.04
Codename:	bionic


#### Check CUDA Version

In [3]:
# Check CUDA/cuDNN Version
!nvcc -V && which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
/usr/local/cuda/bin/nvcc


#### Check GPU Version

In [4]:
# Check GPU
!nvidia-smi

Mon May 13 09:31:40 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    17W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

#### Check GPU if You've Right Version (T4)
Many thanks to NVIDIA team for this snippet of code to automatically set up everything.

In [5]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name != b'Tesla T4':
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


### Installation of cuDF/cuML
Many thanks to NVIDIA team for this snippet of code to automatically set up everything.

In [6]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c conda-forge \
  -c rapidsai-nightly/label/cuda10.0 -c nvidia/label/cuda10.0 \
  cudf cuml

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
  shutil.copy('/usr/local/lib/'+fn, os.getcwd())

--2019-05-13 09:31:42--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M  64.9MB/s    in 0.9s    

2019-05-13 09:31:48 (64.9 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk

## Critical Imports

In [0]:
# Critical imports
import nvstrings, nvcategory, cudf
import cuml
import os
import numpy as np
import pandas as pd

## Creating

### Create a Series of integers

In [25]:
gdf = cudf.Series([1, 2, 3, 4, 5, 6])
print(gdf)
print(type(gdf))

0    1
1    2
2    3
3    4
4    5
5    6
dtype: int64
<class 'cudf.dataframe.series.Series'>


### Create a Series of floats

In [26]:
gdf = cudf.Series([1., 2., 3., 4., 5., 6.])
print(gdf)

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    6.0
dtype: float64


### Create a  Series of strings


In [27]:
gdf = cudf.Series(['a', 'b', 'c'])
print(gdf)

0    a
1    b
2    c
dtype: object


### Create 3 column DataFrame
- Consisting of dates, integers and floats

In [28]:
# Import
import datetime as dt

# Using a list of tuples
# Each element in the list represents a category
# The first element of the tuple is the category's name
# The second element of the tuple is a list of the values in that category
gdf = cudf.DataFrame([
    # Create 10 busindates ess from 1st January 2019 via pandas
    ('dates', pd.date_range('1/1/2019', periods=10, freq='B')),
    # Integers
    ('integers', [i for i in range(10)]),
    # Floats
    ('floats', [float(i) for i in range(10)])
])

# Print dataframe
print(gdf)

                     dates  integers  floats
0 2019-01-01T00:00:00.000         0     0.0
1 2019-01-02T00:00:00.000         1     1.0
2 2019-01-03T00:00:00.000         2     2.0
3 2019-01-04T00:00:00.000         3     3.0
4 2019-01-07T00:00:00.000         4     4.0
5 2019-01-08T00:00:00.000         5     5.0
6 2019-01-09T00:00:00.000         6     6.0
7 2019-01-10T00:00:00.000         7     7.0
8 2019-01-11T00:00:00.000         8     8.0
9 2019-01-14T00:00:00.000         9     9.0


### Create 2 column Dataframe
- Consisting of integers and string category

In [29]:
# Using a list of tuples
# Each element in the list represents a category
# The first element of the tuple is the category's name
# The second element of the tuple is a list of the values in that category
gdf = cudf.DataFrame([
    ('integers', [1 ,2, 3, 4]),
    ('string', ['a', 'b', 'c', 'd'])
])

print(gdf)

   integers  string
0         1       a
1         2       b
2         3       c
3         4       d


### Create a 2 Column  Dataframe with Pandas Bridge
- Consisting of integers and string category
- For all string columns, you must convert them to type `category` for filtering functions to work intuitively (for now)

In [30]:
# Create pandas dataframe
pandas_df = pd.DataFrame({
    'integers': [1, 2, 3, 4], 
    'strings': ['a', 'b', 'c', 'd']
})

# Convert string column to category format
pandas_df['strings'] = pandas_df['strings'].astype('category')

# Bridge from pandas to cudf
gdf = cudf.DataFrame.from_pandas(pandas_df)

# Print dataframe
print(gdf)

   integers  strings
0         1        a
1         2        b
2         3        c
3         4        d


## Viewing

### Printing Column Names

In [31]:
gdf.columns

Index(['integers', 'strings'], dtype='object')

### Viewing Top of DataFrame

In [39]:
num_of_rows_to_view = 2 
print(gdf.head(num_of_rows_to_view))

   integers  strings
0         1        a
1         2        b


### Viewing Bottom of DataFrame

In [40]:
num_of_rows_to_view = 3 
print(gdf.tail(num_of_rows_to_view))

   integers  strings
1         2        b
2         3        c
3         4        d


## Filtering

### Method 1: Query

#### Filtering Integers/Floats by Column Values
- This only works for floats and integers, not for strings

In [32]:
print(gdf.query('integers == 1'))

   integers  strings
0         1        a


#### Filtering Strings by Column Values
- This only works for floats and integers, not for strings so this will return an error!

In [33]:
print(gdf.query('strings == a'))

KeyError: ignored

### Method 2:  Simple Columns

#### Filtering Strings by Column Values


In [34]:
# Filtering based on the string column
print(gdf[gdf.strings == 'b'])

   integers  strings
1         2        b


####Filtering Integers/Floats by Column Values

In [35]:
# Filtering based on the string column
print(gdf[gdf.integers == 2])

   integers  strings
1         2        b


### Method 2:  Simple Rows

#### Filtering by Row Numbers

In [41]:
# Filter rows 0 to 2 (not inclusive of the third row with the index 2)
print(gdf[0:2])

   integers  strings
0         1        a
1         2        b


### Method 3:  loc[rows, columns]

In [46]:
# The syntax is as follows loc[rows, columns] allowing you to choose rows and columns accordingly
# The example allows us to filter the first 3 rows (inclusive) of the column integers
print(gdf.loc[0:2, ['integers']])

   integers
0         1
1         2
2         3
